# TensorFlow based Logistic regression from scratch

Tensorflow is an open source machine-learning (ML) library created by Google. It has predominantly become prevalent because of the extensive Deep Learning algorithms it supports. Aside from that, it supports both the GPU and CPU, it's highly scalable and can run on mobile devices with small memory footprint. 

## Introduction

In this Jupyter notebook I provide an in-depth discussion of the abstract computational principles underlying the TensorFlow software library. I do so, by implementing Logistic Rgeression **without using any of the available TF based Logistic regression methoods**. 

Location of this notebook:
https://github.com/QuantScientist/deep-ml-meetups/tree/master/tensorflow-dl


# Objectives

In this tutorial, you will discover how to implement logistic regression from scratch with using Python and TensorFlow.

After completing this tutorial, you will know:

* How to write a simple program in TensorFlow


In [7]:
# docker run -it -p 8888:8888 -p 6006:6006 -v //Volumes/1tb-ext/2019/db/stan/Dropbox/dev2/books/2019/BML/:/root/sharedfolder floydhub/dl-docker:cpu bash
!pip install --upgrade pip
!pip install --upgrade bokeh
!pip install --upgrade xgboost
!pip install --upgrade watermark
!pip install --upgrade edward tensorflow
!pip install --upgrade seaborn

Requirement already up-to-date: pip in /usr/local/lib/python2.7/dist-packages
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security

# Import Libs

In [8]:
#%reset -f
import numpy
import pandas
import os
import sys
import numpy as np
import pandas as pd
import uuid
import pandas as pd
import itertools
import warnings
# Import Bokeh modules for interactive plotting
# from bokeh import *
import seaborn as sns
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys,inspect

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas
from scipy.stats import norm, invgamma
from sklearn import linear_model
import numpy as np
import scipy.sparse
import pickle
# import xgboost as xgb
import numpy
import os

import numpy
import matplotlib.pyplot as plt
rng = numpy.random

print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
/root/sharedfolder/deep-ml-git/deep-ml-meetups/tensorflow-dl



# Versions

In [9]:
%load_ext watermark
%watermark -d -v -m -p pandas,scipy,matplotlib,tensorflow

2017-02-24 

CPython 2.7.6
IPython 4.2.0

pandas 0.13.1
scipy 0.13.3
matplotlib 1.3.1
tensorflow 1.0.0

compiler   : GCC 4.8.2
system     : Linux
release    : 4.9.8-moby
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


# Setup display

In [10]:
import pandas as pd
import seaborn as sns
import warnings 
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', None)
pd.set_option('display.line_width', 200)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)



# Display graphics in this notebook
# from bokeh import *
# bokeh.io.output_notebook()

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

/usr/lib/python2.7/dist-packages/pandas/core/config.py:570: DeprecationWarning: line_width has been deprecated, use display.width instead (currently both are
identical)

  warnings.warn(d.msg, DeprecationWarning)
/usr/lib/python2.7/dist-packages/pandas/core/config.py:570: DeprecationWarning: height has been deprecated.

  warnings.warn(d.msg, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadat

# Test that TF is working

In [11]:
# %reset -f
import tensorflow as tf
tf.reset_default_graph()
# input images
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")    
hello = tf.constant('Hello, TensorFlow!')

# Start tf session
sess = tf.InteractiveSession()
# sess = tf.Session()
# Run graph
print sess.run(hello)

graph = tf.get_default_graph()
for op in graph.get_operations(): 
    print(op.name)

Hello, TensorFlow!
input/x-input
input/y-input
Const


# Pure Python 

In [12]:
import tensorflow as tf
from scipy.stats import logistic
from scipy.special import expit

# Link Function

def sigmoid(x):
    return logistic.cdf(x) # or expit(x)
# Link Function deriviative 
def ddsigmoid(z):
    return sigmoid(z) * (1.0 - sigmoid(z))

# print logistic.cdf(0.387)
# print expit(0.387)

print ddsigmoid(0.387)


0.240868228634


# TF

In [18]:
sess = tf.InteractiveSession()
tf.reset_default_graph()

one=tf.constant(1.0,"float64") # To avoid: ValueError Tensor conversion requested dtype float32 for 
                               # Tensor with dtype float64: 'Tensor("Exp:0", dtype=float64)'
X = tf.placeholder("float64") # create symbolic variable
Y = tf.placeholder("float64") # create symbolic variable

x_77=tf.constant(0.387)

# derivative of sigmoid= sigmoid(y) * (1.0 - sigmoid(y))

sigmoid=(tf.div(one, (one + tf.exp(-X))))
dsigmoid=tf.multiply(Y, tf.subtract(one,Y)) 
  
init = tf.initialize_all_variables() # you need to initialize variables (in this case just variable W)
sess = tf.Session()
print sess.run(init)

print sess.run(sigmoid, {X:x_77.eval(session=sess)}) 
print sess.run(sigmoid, {X:0.387}) 
print sess.run(dsigmoid,{Y:sigmoid.eval({X:0.387}, session=sess)}) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
None
0.595560300929
0.59556030225
0.240868228634


In [44]:
show_graph(tf.get_default_graph().as_graph_def())
writer = tf.train.SummaryWriter("tf_persistent.pbtxt", graph=tf.get_default_graph())

# gdef = tf.GraphDef()
# from google.protobuf import text_format
# text_format.Merge(open("tf_persistent.pbtxt").read(), gdef)
# show_graph(gdef)

AttributeError: 'module' object has no attribute 'SummaryWriter'

# References

https://arxiv.org/pdf/1611.08903.pdf

https://arxiv.org/pdf/1610.01178.pdf

https://gist.github.com/msakai/2b3dbcc64ae1faefea30

https://www.kaggle.com/dineshrai/d/uciml/pima-indians-diabetes-database/log-reg-vs-neural-network/code

http://killianlevacher.github.io/blog/posts/post-2016-07-22/post.html
